# Getting Started with the Snowflake Python API

The Snowflake Python API allows you to manage Snowflake using Python. Using the API, you're able to create, delete, and modify tables, schemas, warehouses, tasks, and much more, in many cases without needing to write SQL or use the Snowflake Connector for Python. 

In this tutorial, we show how you can use the Snowflake API to create objects in Snowflake *completely in Python*. Not a single line of SQL required!

This tutorial is based on [this quickstart](https://quickstarts.snowflake.com/guide/getting-started-snowflake-python-api/index.html), which includes more in-depth overview of the Snowflake Python API and additional learning modules not covered in this notebook.

**Requirements:** Please add the `snowflake` package from the package picker on the top right. We will be using this packages in the notebook.

In [ ]:
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.warehouse import Warehouse

With notebooks, you can use the `get_active_session()` command to get a session object to work with. No need to specify any connection parameters! 

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

Then, we create a `Root` object to use the API’s types and methods.

In [ ]:
api_root = Root(session)  

## Create a database, schema, and table
Let's use our `api_root` object to create a database, schema, and table in your Snowflake account.

Create a database and schema by running the following cell in the notebook:

In [ ]:
database_ref = api_root.databases.create(Database(name="demo_database"), mode="orreplace")

In [ ]:
schema_ref = database_ref.schemas.create(Schema(name="demo_schema"), mode="orreplace")

By looking at the queries in your Query History, you can see that this is the corresponding SQL query: 
```sql
CREATE OR REPLACE SCHEMA DEMO_DATABASE.DEMO_SCHEMA;
```

Now let's create a demo table with two sample columns.

In [ ]:
table_ref = schema_ref.tables.create(Table(
        name="demo_table", 
        columns=[TableColumn(name="c1",datatype="int",nullable=False), TableColumn(name="c2", datatype="string")]
    ), mode="orreplace")

SQL equivalent to the Python command above: 
```sql
CREATE OR REPLACE table DEMO_DATABASE.DEMO_SCHEMA.DEMO_TABLE  (C1 int not null ,C2 string );
```


## Retrieve object data
Let's cover a couple of ways to retrieve metadata about an object in Snowflake. Run the following cell to look at the documentation for this method: 

In [ ]:
table_ref.fetch

In [ ]:
demo_table = table_ref.fetch()

In [ ]:
demo_table.to_dict()

## Programmatically update a table

Now let's append one additional column to this table declaratively. Then, we use this to update the table.

In [ ]:
demo_table.columns.append(TableColumn(name="c3", datatype="int", nullable=False, constraints=[PrimaryKey()]))

Now, we see that the C3 column has been added. 

In [ ]:
demo_table.to_dict()

## Create, suspend, and delete a warehouse

We can also create a small warehouse using the API.

In [ ]:
# create a warehouse collection
warehouses = api_root.warehouses
# create a Warehouse instance that used to store the property of a warehouse
warehouse_name = "WH_DEMO"
warehouse_demo = Warehouse(
    name=warehouse_name,
    warehouse_size="SMALL",
    auto_suspend=500,
)
# create a warehouse and retrive its reference
warehouse_ref = warehouses.create(warehouse_demo)

In [ ]:
# Fetch warehouse details.
warehouse = warehouse_ref.fetch()
warehouse.to_dict()

We can search through all the warehouses currently available.

In [ ]:
warehouse_list = warehouses.iter(like=warehouse_name)
result = next(warehouse_list)
result.to_dict()

We can change the size of the warehouse from `SMALL` to `LARGE`.

In [ ]:
# Update the warehouse. Change it's size to LARGE
warehouse_ref.create_or_update(Warehouse(
    name=warehouse_name,
    warehouse_size="LARGE",
    auto_suspend=500,
))

We can check the updated warehouse size: 

In [ ]:
# Check the warehouse 
warehouse_ref.fetch().size

Finally, we can delete the warehouse once we are done using it.

In [ ]:
# Delete the warehouse
warehouse_ref.delete()

## Conclusion

In this Quickstart, you learned the fundamentals for managing Snowflake objects using the Snowflake Python API. To learn more about the Snowflake Python, see 
[Snowflake Documentation](https://docs.snowflake.com/developer-guide/snowflake-python-api/snowflake-python-overview?_fsi=mOxvauSe&_fsi=mOxvauSe).
